In [ ]:
#Tool to make labelling of the data easier.
#GUI explanation:
#Auto-truncate/Auto all: truncate the current/all samples with the label algorithm
#Slider: use the slider to label by hand, Truncate all needs to be pressed to apply the changes!!!
#Save: save the samples object as a json file
#Prev/Next: navigate through the samples

In [ ]:
#IMPORTS
%matplotlib notebook

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, interact
from IPython.display import display
import samples as ls
import model_types

load_test = True

load_samples = True
model_type = 1
accel,gyro,compass = 'xyz','xyz',''
DIM = len(accel)+len(gyro)+len(compass)

In [ ]:
#LOAD DATA - MONGO VARIANT

labels,sample_ids,filename = model_types.get_labels(model_type, load_test)

straight = 0
label_index = []
dataset = []

#not relevant for angle tasks
if model_type < 10:
    #data for auto truncating
    #weights used when auto truncating
    weights_dict = {'straight':  [1,0,0, 0,0,0, 0,0,0],
                    'curve_s_l': [0,0,0, 0,0,1, 0,0,0],
                    'curve_s_r': [0,0,0, 0,0,1, 0,0,0],
                    'curve_m_l': [0,0,0, 0,0,1, 0,0,0],
                    'curve_m_r': [0,0,0, 0,0,1, 0,0,0],
                    'curve_l_l': [0,0,0, 0,0,1, 0,0,0],
                    'curve_l_r': [0,0,0, 0,0,1, 0,0,0],
                    'curve_ml_r': [0,0,0, 0,0,1, 0,0,0],
                    'curve_sm_r': [0,0,0, 0,0,1, 0,0,0],
                    'curve_ml_l': [0,0,0, 0,0,1, 0,0,0],
                    'curve_sm_l': [0,0,0, 0,0,1, 0,0,0],
                    'wall': [0,1,0, 0,0,0, 0,0,0],
                    'foot': [0.9,0.1,0, 0,0,0, 0,0,0],
                    'object_l': [0,0,0, 0,1,0, 0,0,0],
                    'object_r': [0,0,0, 0,0,1, 0,0,0],
                    'angle': [1./6.,1./6.,1./6., 1./6.,1./6.,1./6., 0,0,0]
                   }
    
    for key in weights_dict:
        w = []
        if 'x' in accel: w.append(weights_dict[key][0])
        if 'y' in accel: w.append(weights_dict[key][1])
        if 'z' in accel: w.append(weights_dict[key][2])
        if 'x' in gyro: w.append(weights_dict[key][3])
        if 'y' in gyro: w.append(weights_dict[key][4])
        if 'z' in gyro: w.append(weights_dict[key][5])
        if 'x' in compass: w.append(weights_dict[key][6])
        if 'y' in compass: w.append(weights_dict[key][7])
        if 'z' in compass: w.append(weights_dict[key][8])
        weights_dict[key] = w

    offset_dict = {'straight': 0,
                    'curve_s_l': 5,
                    'curve_s_r': 5,
                    'curve_m_l': 5,
                    'curve_m_r': 5,
                    'curve_l_l': 5,
                    'curve_l_r': 5,
                    'curve_ml_l': 5,
                    'curve_sm_l': 5,
                    'curve_ml_r': 5,
                    'curve_sm_r': 5,
                    'wall': 5,
                    'foot': 5,
                    'object_l': 10,
                    'object_r': 10,
                    'angle': 0
                   }
    duration_dict = {'straight': 0,
                    'curve_s_l': 0,
                    'curve_s_r': 0,
                    'curve_m_l': 0,
                    'curve_m_r': 0,
                    'curve_l_l': 0,
                    'curve_l_r': 0,
                    'curve_ml_l': 0,
                    'curve_sm_l': 0,
                    'curve_ml_r': 0,
                    'curve_sm_r': 0,
                    'wall': 30,
                    'foot': 50,
                    'object_l': 50,
                    'object_r': 50,
                    'angle': 0
                   }
    percent_dict = {'straight': 0.4,
                    'curve_s_l': 0.4,
                    'curve_s_r': 0.4,
                    'curve_m_l': 0.4,
                    'curve_m_r': 0.4,
                    'curve_l_l': 0.4,
                    'curve_l_r': 0.4,
                    'curve_ml_l': 0.4,
                    'curve_sm_l': 0.4,
                    'curve_ml_r': 0.4,
                    'curve_sm_r': 0.4,
                    'wall': 0.8,
                    'foot': 0.8,
                    'object_l': 0.8,
                    'object_r': 0.8,
                    'angle': 0.4
                   }
    for l in labels:
        if l == 'straight': continue
        assert np.isclose(np.sum(weights_dict[l]),1)

if load_samples:
    samples = ls.load(filename)
    #samples = ls.load('data_2.json')
    #samples.remove_label('foot')
else:
    samples = ls.Samples(labels)

    for label in labels:
        samples.load_samples(label = label, sample_ids = sample_ids)

    samples.convert_to_input(size=None, accel=accel, gyro=gyro, compass=compass)

cur_index = 0
dataset = []
dataset_y = []
index_reverse = []
trunc_index = []
trunc_labels = samples.labels_data
for label in labels:
    data = samples.data[label]
    labels_data = samples.labels_data[label]
    
    assert len(data) == len(labels_data)
    for i in range(len(data)):
        dataset.append(data[i])
        index_reverse.append(i)
    
        first = 0
        last = len(labels_data[i])-1
        #assert len(labels_data[i]) == len(data[i])
        if load_samples:
            found = False
            class_id = 0
            for k in range(len(labels_data[i])):
                d = labels_data[i][k]
                if d != 0 and not found: #anything else -> class found
                    first = k
                    class_id = d
                    found = True
                elif d==0 and found:
                    last = k-1
                    break;
            assert labels[class_id] == label
                
        dataset_y.append(samples.labels_dict[label])
        trunc_index.append({'x': first,'y': last})
        

In [ ]:
if True:
    for key in labels:
        print('%s: %s / %s' % (key, len(samples.data[key]), len(samples.test[key])))

In [ ]:
def get_label(index):
    if index < len(dataset_y):
        return labels[dataset_y[index]]
    raise ValueError('Unknown index %s' % index)

#window = for moving average
def smooth(data, window=10, steps=1):
    #if len(data) < window: raise ValueError('Window too small')
    
    #important: use np.arrays, since only then += is an element wise operation
    data = np.array(data)
    if window > 1:
        data2 = []
        for i in range(len(data)-window):
            data2.append(np.array([0.0]*DIM))
            for m in range(window):
                data2[i] += data[i+m]
            data2[i] /= window
        data = data2
    
    for k in range(steps):
        data2 = []
        for i in range(len(data)-2):
            data2.append((data[i] + 2*data[i+1] + data[i+2])/4)
        data = data2
    return data
    
def calc_trunc(index):
    window = 10
    steps = 2
    data = dataset[index]
    last_index = len(data)-1
    data = smooth(data, window, steps)
    weights = weights_dict[get_label(index)]
    offset = offset_dict[get_label(index)]
    duration = duration_dict[get_label(index)]
    percent = percent_dict[get_label(index)]
    
    data2 = data#[offset:]
    data = []
    for i in range(len(data2)):
        data.append( np.array([0]*DIM) )
    
    max_d = np.array([0.0]*DIM)
    for i in range(len(data2)-1):
        d = np.abs(data2[i+1]-data2[i])
        for dim in range(DIM):
            if d[dim]>max_d[dim]: max_d[dim] = d[dim]
    
    max_d = max_d*percent
    data[0] = data2[0]
    for i in range(1,len(data2)):
        d = np.abs(data2[i]-data2[i-1])
        for dim in range(DIM):
            if d[dim]>=max_d[dim]:
                data[i][dim] = 1 #+data[i-1]
            else:
                data[i][dim] = 0 #+data[i-1]
    
    found = [False]*DIM
    first = [0]*DIM
    last = [len(data)-1]*DIM
    for dim in range(DIM):
        for i in range(len(data)):
            if data[i][dim] == 1:
                if not found[dim]:
                    first[dim] = i
                    found[dim] = True
                else: 
                    last[dim] = i
    
    first = int(.5 + np.average(first, weights=weights))
    last = int(.5 + np.average(last, weights=weights))
    first = first+offset+window+steps
    last = last+offset+window+steps
    
    last = min(last, last_index)
    first = min(first,last)
    min_last = min(first+duration, last_index)
    last = max(last, min_last)
    
    return first,last


def derive(data):
    data2 = []
    for i in range(len(data)-10):
        d = np.abs(data[i+10]-data[i])
        data2.append(d)
    return data2

def truncs(data):
    max_d = np.array([0.0]*DIM)
    for i in range(len(data)-10):
        d = np.abs(data[i+10]-data[i])
        for dim in range(DIM):
            if d[dim]>max_d[dim]: max_d[dim] = d[dim]
    data2 = data[:]
    max_d = max_d*0.8
    for i in range(10,len(data2)):
        d = np.abs(data2[i]-data2[i-10])
        for dim in range(DIM):
            if d[dim]>max_d[dim]:
                data[i][dim] = 1 #+data[i-1]
            else:
                data[i][dim] = 0 #+data[i-1]
    return data
    
#for i in range(len(dataset)):
    #dataset[i] = smooth(dataset[i], 10, 0)
    #dataset[i] = derive(dataset[i])
    #dataset[i] = truncs(dataset[i])

In [ ]:
#WIDGET INIT

btnOk = widgets.Button(description='Truncate all')
def truncate(b):
    for index in range(len(dataset)):
        if get_label(index) == 'straight': continue
    
        x = trunc_index[index]['x']
        y = trunc_index[index]['y']
        data = dataset[index]

        new_labels = [0]*len(data)
        for i in range(x,y+1):
            new_labels[i] = dataset_y[index]

        trunc_labels[get_label(index)][index_reverse[index]] = new_labels
    set_data(cur_index)
btnOk.on_click(truncate)

btnAuto = widgets.Button(description='Auto truncate')
def auto_trunc(b):
    #algorithm doesnt work for straight
    if get_label(cur_index) == 'straight': return
    
    x,y = calc_trunc(cur_index)
    trunc_index[cur_index]['x'] = x
    trunc_index[cur_index]['y'] = y
    data = dataset[cur_index]
    
    new_labels = [0]*len(data)
    for i in range(x,y+1):
        new_labels[i] = dataset_y[cur_index]
    
    trunc_labels[get_label(cur_index)][index_reverse[cur_index]] = new_labels
    set_data(cur_index)
btnAuto.on_click(auto_trunc)

btnAutoAll = widgets.Button(description='Auto all')
def auto_all(b):
    for index in range(len(dataset)):
        if get_label(index) == 'straight': continue
    
        x,y = calc_trunc(index)
        trunc_index[index]['x'] = x
        trunc_index[index]['y'] = y
        data = dataset[index]

        new_labels = [0]*len(data)
        for i in range(x,y+1):
            new_labels[i] = dataset_y[index]

        trunc_labels[get_label(index)][index_reverse[index]] = new_labels
    set_data(cur_index)
btnAutoAll.on_click(auto_all)


btnReset = widgets.Button(description='Reset')
def reset(b):
    trunc_index[cur_index] = {'x': 0, 'y': len(dataset[cur_index])-1}
    set_data(cur_index)
btnReset.on_click(reset)

btnSave = widgets.Button(description='Save')
def save(b):
    ls.save(samples, filename)
    print('Saved! (%s)' % filename)
btnSave.on_click(save)


trunc_sl = widgets.IntRangeSlider(value=[0,1], min=0, max=trunc_index[cur_index]['y'])
def update_trunc(change):
    x = change['new'][0]
    y = change['new'][1]
    trunc_index[cur_index] = {'x': x, 'y': y}
    draw_data_cutter()
trunc_sl.observe(update_trunc, names='value')

txtIndex = widgets.Text(value=str(cur_index))
def update_index(sender):
    try:
        index = int(txtIndex.value)
        set_data(index)
    except ValueError:
        txtIndex.value = str(cur_index)
txtIndex.on_submit(update_index)

txtLabel = widgets.Text(value=labels[0],disabled=True)
    
btnPrev = widgets.Button(description='Prev')
def prev_data(b):
    global cur_index
    cur_index -= 1
    if(cur_index < 0): cur_index = 0
    else: set_data(cur_index)
btnPrev.on_click(prev_data)
btnNext = widgets.Button(description='Next')
def next_data(b):
    global cur_index
    cur_index += 1
    if(cur_index >= len(dataset)): cur_index = len(dataset)-1
    else: set_data(cur_index)
btnNext.on_click(next_data)

def set_data(index):
    global cur_index 
    cur_index = index
    txtIndex.value = str(index)
    txtLabel.value = get_label(index)
    x = trunc_index[index]['x']
    y = trunc_index[index]['y']
    trunc_sl.max = len(dataset[index])-1
    trunc_sl.value = [x,y]
    draw_data_cutter()

In [ ]:
#PLOT
fig = plt.figure()
ax = fig.add_subplot(111)
plt.ion()
fig.show()
fig.canvas.draw()

def draw_data_cutter():
    x = trunc_index[cur_index]['x']
    y = trunc_index[cur_index]['y']
    ax.clear()
    
    data2 = np.array(dataset[cur_index])
    #maybe smooth for display
    #data2 = smooth(data2, 10, 2)
    
    #transform (size, 6) -> (6, size, 1) (for draw each xyzxyz separately)
    data = []
    for i in range(DIM): data.append([])
    for d in data2:
        for i in range(DIM):
            data[i].append(d[i])
        
    for i in range(DIM):
        if i<3 and DIM>3:
            label = 'accel %s' % ('xyz'[i])
        else:
            label = 'gyro %s' % ('xyz'[i-3])
        ax.plot(range(len(data[i])),data[i], label=label)
        
    ax.axvspan(x, y, color='green', alpha=0.1)
    ax.legend(loc='lower left')
    fig.canvas.draw()

display(widgets.HBox((btnOk,btnAuto,btnAutoAll,btnReset,btnSave)))
display(widgets.HBox((btnPrev,btnNext,txtIndex,txtLabel)))
display(trunc_sl)
set_data(cur_index)

In [ ]:
#Save the graph as image
if False: plt.savefig('data-index%s.png' % cur_index,pad_inches=0,bbox_inches='tight')